## Структура
В этом ноутбуке проверка и исследование класса `Surface` и его методов.

In [1]:
import numpy as np
import math
import draw

from common import *
from surface import Surface

%matplotlib inline

In [2]:
dataset = 'datasets/teddy.obj'   # ADJUST

Проверенный датасеты:
- simplex (все ок)
- teddy (все ок, кроме Шлафли)
- dragon (не прорисовывались; Шлафли: 0.6140038130661708, -6826.831639349427, -497.1132568607541)

In [3]:
surface = Surface.from_obj(dataset)
surface.draw()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='#cccccc', intensity=0.5, position=(-3.0, 5…

In [5]:
print(f'Volume: {surface.volume()}')

print()

angle_defects_sum = sum(surface.angle_defects())
euler_2_pi = 2 * math.pi * surface.Euler()
print(f'Angle defects sum: {angle_defects_sum}')
print(f'Euler characteristic: (V - E + F) * 2 * pi = {euler_2_pi}')
diff = angle_defects_sum - euler_2_pi
print(f'diff: {diff}, close to 0: {math.isclose(diff, 0, abs_tol=EPS)}')

Volume: 8679.564581153883

Angle defects sum: 12.566370614358958
Euler characteristic: (V - E + F) * 2 * pi = 12.566370614359172
diff: -2.149391775674303e-13, close to 0: True


По дискретной теореме __Гаусса-Бонне__: $\sum{d(v)} = 2\pi\chi(M)$

In [6]:
step = 0.1
for i in range(10):
    surface.coordinates += np.array(surface.mean_curvature_flow()) * step

In [7]:
surface.draw()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='#cccccc', intensity=0.5, position=(-3.0, 5…

Выпуклые части должны надуваться, у мишки их много: голова, руки, конечности. Они друг на друга налазят, поэтому ему немного плохо. Для проверки можно заменить значения потока на противоположные -- тогда он начнет тощать и становится плоским.

In [8]:
step = 0.1
for i in range(20):
    surface.coordinates -= np.array(surface.mean_curvature_flow()) * step
surface.draw()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='#cccccc', intensity=0.5, position=(-3.0, 5…

__Schlafli Formula__: $\sum_{i, j \in E}{l_{ij} \frac{d}{dt} \phi_{ij}} = 0$, где $l_{ij}$ - длина ребра, $\phi_{ij}$ - плоский угол между гранями.

In [9]:
import random

flows = {
    'random < 0.5': [np.array([random.randrange(0, 1e3) * 1e-6 for _ in range(3)]) for _ in range(surface.V)],
    'random < 1': [np.array([random.randrange(0, 1e9) * 1e-9 for _ in range(3)]) for _ in range(surface.V)],
    'mean_curvature_flow * 0.1': np.array(surface.mean_curvature_flow()) * 0.1,
}

for name, flow in flows.items():
    print(f'Schlafli, {name}: {surface.schlafli(flow)}')

Schlafli, random < 0.5: 1.305413074620572e-09
Schlafli, random < 1: 0.024684698929371862
Schlafli, mean_curvature_flow * 0.1: 0.0005740115409285743


Для больших значений потока значение далеко от 0. Наверное стоит как-то по умному выбирать _t_ для поиска производной/придумывать что-то с погрешностью.